In [1]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
# Analyzes regression in app benchmark and prints relative differences in appRegression.csv

start = 0.05
end = 0.2

folder = "../../results_all/app_history_influxdb/"
filename = "../../results_aggr/app_history_influxdb.csv"
print(folder)

../../results_all/app_history_influxdb/


In [3]:
rows = []
all_runs = glob.glob(folder + "*")
for fullRunFolder in all_runs:
    runFolder = fullRunFolder + "/"
    all_numbers = glob.glob(runFolder + "*")
    for fullNumberFolder in all_numbers:

        all_files = glob.glob(fullNumberFolder + "/*")
        size = len(all_files)

        if size != 17:
            print(f"{size} elements in folder {fullNumberFolder}, skip.")
        else:
            tmp = fullNumberFolder.split("\\")

            #Parse run and commit number from path structure
            run = tmp[len(tmp)-2]
            number = tmp[len(tmp)-1]
            print(f"Run {run} Number {number}")

            #Read latency file for both types (old and new)
            insertsOld = pd.read_csv(fullNumberFolder + "/latenciesInsertsOld.csv",
                             header=None,
                             names=["worker","batch","latency (ms)"])
            n = len(insertsOld.index)
            removeFirst = int(start * n)
            removeLast = int(end * n)
            insertsOld = insertsOld.iloc[removeFirst:-removeLast , :]

            insertsNew = pd.read_csv(fullNumberFolder + "/latenciesInsertsNew.csv",
                             header=None,
                             names=["worker","batch","latency (ms)"])
            n = len(insertsNew.index)
            removeFirst = int(start * n)
            removeLast = int(end * n)
            insertsNew = insertsNew.iloc[removeFirst:-removeLast , :]

            queries1Old = pd.read_csv(fullNumberFolder + "/latenciesQueries1Old.csv",
                             header=None,
                             names=["query","latency (ms)"])
            n = len(queries1Old.index)
            removeFirst = int(start * n)
            removeLast = int(end * n)
            queries1Old = queries1Old.iloc[removeFirst:-removeLast , :]

            queries2Old = pd.read_csv(fullNumberFolder + "/latenciesQueries2Old.csv",
                             header=None,
                             names=["query","latency (ms)"])
            n = len(queries2Old.index)
            removeFirst = int(start * n)
            removeLast = int(end * n)
            queries2Old = queries2Old.iloc[removeFirst:-removeLast , :]

            queries1New = pd.read_csv(fullNumberFolder + "/latenciesQueries1New.csv",
                             header=None,
                             names=["query","latency (ms)"])
            n = len(queries1New.index)
            removeFirst = int(start * n)
            removeLast = int(end * n)
            queries1New = queries1New.iloc[removeFirst:-removeLast , :]

            queries2New = pd.read_csv(fullNumberFolder + "/latenciesQueries2New.csv",
                             header=None,
                             names=["query","latency (ms)"])
            n = len(queries2New.index)
            removeFirst = int(start * n)
            removeLast = int(end * n)
            queries2New = queries2New.iloc[removeFirst:-removeLast , :]

            #Find median latency for both types
            medianInsertsOld = insertsOld["latency (ms)"].median()
            medianInsertsNew = insertsNew["latency (ms)"].median()

            medianQueries1Old = queries1Old["latency (ms)"].median()
            medianQueries1New = queries1New["latency (ms)"].median()

            medianQueries2Old = queries2Old["latency (ms)"].median()
            medianQueries2New = queries2New["latency (ms)"].median()

            #Calculate diff
            diffInserts = ((medianInsertsNew / medianInsertsOld) - 1) * 100
            diffQueries1 = ((medianQueries1New / medianQueries1Old) - 1) * 100
            diffQueries2 = ((medianQueries2New / medianQueries2Old) - 1) * 100

            #Append to rows
            rows.append({
                "run" : run,
                "number" : int(number),
                "diffInserts": diffInserts,
                "diffQueries1": diffQueries1,
                "diffQueries2": diffQueries2
            })

Run run1 Number 1
Run run1 Number 10
Run run1 Number 100
Run run1 Number 101
Run run1 Number 102
Run run1 Number 103
Run run1 Number 104
Run run1 Number 105
Run run1 Number 106
Run run1 Number 107
Run run1 Number 108
Run run1 Number 109
Run run1 Number 11
Run run1 Number 110
Run run1 Number 111
Run run1 Number 12
Run run1 Number 13
Run run1 Number 14
Run run1 Number 15
Run run1 Number 16
Run run1 Number 17
Run run1 Number 18
Run run1 Number 19
Run run1 Number 2
Run run1 Number 20
Run run1 Number 21
Run run1 Number 22
Run run1 Number 23
Run run1 Number 24
Run run1 Number 25
Run run1 Number 26
Run run1 Number 27
Run run1 Number 28
Run run1 Number 29
Run run1 Number 3
Run run1 Number 30
Run run1 Number 31
Run run1 Number 32
Run run1 Number 33
Run run1 Number 34
Run run1 Number 35
Run run1 Number 36
Run run1 Number 37
Run run1 Number 38
Run run1 Number 39
Run run1 Number 4
Run run1 Number 40
Run run1 Number 41
Run run1 Number 42
Run run1 Number 43
Run run1 Number 44
Run run1 Number 45
Run 

In [4]:
# Convert to data frame
df_result = pd.DataFrame(rows)
df_result.sort_values(by=["number"], inplace=True)
df_result.describe()


,number,diffInserts,diffQueries1,diffQueries2
count,314.000000,314.000000,314.000000,314.000000
mean,54.127389,8.370437,-6.950188,-1.022270
std,31.745841,7.407600,7.704671,4.926626
min,1.000000,-1.169412,-29.841361,-18.682129
25%,27.000000,0.215010,-8.132925,-0.492457
50%,53.000000,13.983827,-6.558724,0.321150
75%,79.000000,14.987142,-1.297458,1.224865
max,111.000000,21.672315,2.834186,2.940749


In [5]:
df_result.to_csv(filename)